#  Demo with Fully Convolutional Net

This notebook shows how to load and use **Fully Convolutional Net** from **rtorchn**.

The network implementation can be found [here](https://github.com/TwentyBN/20bn-rtorchn/blob/master/src/main/python/rtorchn/core/networks/smtsmt.py).
    

When trained on something-something (version: 27/06/2017), this network achieves **~30% test accuracy**.

### Imports

In [13]:
import io
import base64
import glob
import os

import gzip
import json
import numpy as np
import torch
from IPython.display import HTML
from skvideo.io import FFmpegReader, ffprobe, vwrite
from torch.autograd import Variable

### Tool to deal with mpeg videos

In [3]:
def show_video(video_filenames):
    """
    Tool to display videos inside the notebook.
    """
    
    if type(video_filenames) is not list:
        video_filenames = [video_filenames]
    
    html_code = ''
    for filename in video_filenames:
        video = io.open(filename, 'r+b').read()
        encoded = base64.b64encode(video)
        html_code += '''
        <video alt="test" width="640" height="480" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        </video>
        '''.format(encoded.decode('ascii'))
        
    return HTML(data= html_code)


def open_mpeg_video(fname, framerate, size):
    """
    Open an mpeg video, and return it as a numpy array.
    """
    
    metadata = ffprobe(fname)
    duration = float(metadata['video']['@duration'])
    # Compute corresponding nb of frames
    nframes = int(duration * framerate)
    oargs = {
        "-r": "%d" % framerate,
        "-vframes": "%d" % nframes,
        "-s": "%dx%d" % (size[0], size[1])
    }
    # Open file
    reader = FFmpegReader(fname, inputdict={}, outputdict=oargs)
    video = []
    # Get frames until there is no more
    for frame in reader.nextFrame():
        video.append(frame)
    # Return as a numpy array
    return np.array(video)

### Mapping Class index <-> Class string

In [4]:
int2label = {
     0: 'Approaching [something] with your camera',
     1: 'Attaching [something] to [something]',
     2: 'Bending [something] so that it deforms',
     3: 'Bending [something] until it breaks',
     4: 'Burying [something] in [something]',
     5: 'Closing [something]',
     6: 'Covering [something] with [something]',
     7: 'Digging [something] out of [something]',
     8: 'Dropping [something] behind [something]',
     9: 'Dropping [something] in front of [something]',
     10: 'Dropping [something] into [something]',
     11: 'Dropping [something] next to [something]',
     12: 'Dropping [something] onto [something]',
     13: 'Failing to put [something] into [something] because [something] does not fit',
     14: 'Folding [something]',
     15: 'Hitting [something] with [something]',
     16: 'Holding [something]',
     17: 'Holding [something] behind [something]',
     18: 'Holding [something] in front of [something]',
     19: 'Holding [something] next to [something]',
     20: 'Holding [something] over [something]',
     21: 'Laying [something] on the table on its side, not upright',
     22: 'Letting [something] roll along a flat surface',
     23: 'Letting [something] roll down a slanted surface',
     24: 'Letting [something] roll up a slanted surface, so it rolls back down',
     25: 'Lifting [something] up completely without letting it drop down',
     26: 'Lifting [something] up completely, then letting it drop down',
     27: 'Lifting [something] with [something] on it',
     28: 'Lifting a surface with [something] on it but not enough for it to slide down',
     29: 'Lifting a surface with [something] on it until it starts sliding down',
     30: 'Lifting up one end of [something] without letting it drop down',
     31: 'Lifting up one end of [something], then letting it drop down',
     32: 'Moving [part] of [something]',
     33: 'Moving [something] across a surface until it falls down',
     34: 'Moving [something] across a surface without it falling down',
     35: 'Moving [something] and [something] away from each other',
     36: 'Moving [something] and [something] closer to each other',
     37: 'Moving [something] and [something] so they collide with each other',
     38: 'Moving [something] and [something] so they pass each other',
     39: 'Moving [something] away from [something]',
     40: 'Moving [something] away from the camera',
     41: 'Moving [something] closer to [something]',
     42: 'Moving [something] down',
     43: 'Moving [something] towards the camera',
     44: 'Moving [something] up',
     45: 'Moving away from [something] with your camera',
     46: 'Opening [something]',
     47: 'Picking [something] up',
     48: 'Piling [something] up',
     49: 'Plugging [something] into [something]',
     50: 'Plugging [something] into [something] but pulling it right out as you remove your hand',
     51: 'Poking [something] so it slightly moves',
     52: 'Poking [something] so lightly that it doesn’t or almost doesn’t move',
     53: 'Poking [something] so that it falls over',
     54: 'Poking [something] so that it spins around',
     55: 'Poking a hole into [some substance]',
     56: 'Poking a hole into [something soft]',
     57: 'Poking a stack of [something] so the stack collapses',
     58: 'Poking a stack of [something] without the stack collapsing',
     59: 'Pouring [something] into [something]',
     60: 'Pouring [something] into [something] until it overflows',
     61: 'Pouring [something] onto [something]',
     62: 'Pouring [something] out of [something]',
     63: 'Pretending or failing to wipe [something] off of [something]',
     64: 'Pretending or trying and failing to twist [something]',
     65: 'Pretending to be tearing [something that is not tearable]',
     66: 'Pretending to close [something] without actually closing it',
     67: 'Pretending to open [something] without actually opening it',
     68: 'Pretending to pick [something] up',
     69: 'Pretending to poke [something]',
     70: 'Pretending to pour [something] out of [something], but [something] is empty',
     71: 'Pretending to put [something] behind [something]',
     72: 'Pretending to put [something] into [something]',
     73: 'Pretending to put [something] next to [something]',
     74: 'Pretending to put [something] on a surface',
     75: 'Pretending to put [something] onto [something]',
     76: 'Pretending to put [something] underneath [something]',
     77: 'Pretending to scoop [something] up with [something]',
     78: 'Pretending to spread “air” onto [something]',
     79: 'Pretending to sprinkle "air" onto [something]',
     80: 'Pretending to squeeze [something]',
     81: 'Pretending to take [something] from [somewhere]',
     82: 'Pretending to take [something] out of [something]',
     83: 'Pretending to throw [something]',
     84: 'Pretending to turn [something] upside down',
     85: 'Pulling [something] from behind of [something]',
     86: 'Pulling [something] from left to right',
     87: 'Pulling [something] from right to left',
     88: 'Pulling [something] onto [something]',
     89: 'Pulling [something] out of [something]',
     90: 'Pulling two ends of [something] but nothing happens',
     91: 'Pulling two ends of [something] so that it gets stretched',
     92: 'Pulling two ends of [something] so that it separates into two pieces',
     93: 'Pushing [something] from left to right',
     94: 'Pushing [something] from right to left',
     95: 'Pushing [something] off of [something]',
     96: 'Pushing [something] onto [something]',
     97: 'Pushing [something] so it spins',
     98: "Pushing [something] so that it almost falls off but doesn't",
     99: 'Pushing [something] so that it falls off the table',
     100: 'Pushing [something] so that it slightly moves',
     101: 'Pushing [something] with [something]',
     102: 'Putting [number of] [something] onto [something]',
     103: 'Putting [something similar to other things that are already on the table]',
     104: 'Putting [something that cannot actually stand upright] upright on the table, so it falls on its side',
     105: 'Putting [something] and [something] on the table',
     106: 'Putting [something] behind [something]',
     107: 'Putting [something] in front of [something]',
     108: 'Putting [something] into [something]',
     109: 'Putting [something] next to [something]',
     110: 'Putting [something] on a flat surface without letting it roll',
     111: 'Putting [something] on a surface',
     112: 'Putting [something] on the edge of [something] so it is not supported and falls down',
     113: 'Putting [something] onto [something else that cannot support it] so it falls down',
     114: 'Putting [something] onto [something]',
     115: 'Putting [something] onto a slanted surface but it doesn’t glide down',
     116: 'Putting [something] that can’t roll onto a slanted surface, so it slides down',
     117: 'Putting [something] that can’t roll onto a slanted surface, so it stays where it is',
     118: 'Putting [something] underneath [something]',
     119: 'Putting [something] upright on the table',
     120: 'Putting [something], [something] and [something] on the table',
     121: 'Removing [something], revealing [something] behind',
     122: 'Rolling [something] on a flat surface',
     123: 'Scooping [something] up with [something]',
     124: 'Show a shadow of [something] that is moving. ',
     125: 'Show a shadow of [something], making sure the shadow is not moving. ',
     126: 'Showing [something] behind [something]',
     127: 'Showing [something] next to [something]',
     128: 'Showing [something] on top of [something]',
     129: 'Showing [something] to the camera',
     130: 'Showing a photo of [something] to the camera',
     131: 'Showing a shadow of [something] that is moving.',
     132: 'Showing a shadow of [something], making sure the shadow is not moving.',
     133: 'Showing that [something] is empty',
     134: 'Showing that [something] is inside [something]',
     135: 'Spilling [something] behind [something]',
     136: 'Spilling [something] next to [something]',
     137: 'Spilling [something] onto [something]',
     138: 'Spinning [something] so it continues spinning',
     139: 'Spinning [something] that quickly stops spinning',
     140: 'Spreading [something] onto [something]',
     141: 'Sprinkling [something] onto [something]',
     142: 'Squeezing [something]',
     143: 'Stacking [number of] [something]',
     144: 'Stuffing [something] into [something]',
     145: 'Taking [one of many similar things on the table]',
     146: 'Taking [something] from [somewhere]',
     147: 'Taking [something] out of [something]',
     148: 'Tearing [something] into two pieces',
     149: 'Tearing [something] just a little bit',
     150: 'Throwing [something]',
     151: 'Throwing [something] against [something]',
     152: 'Throwing [something] in the air and catching it',
     153: 'Throwing [something] in the air and letting it fall',
     154: 'Throwing [something] onto a surface',
     155: 'Tilting [something] with [something] on it slightly so it doesn’t fall down',
     156: 'Tilting [something] with [something] on it until it falls off',
     157: 'Tipping [something] over',
     158: 'Tipping [something] with [something in it] over, so [something in it] falls out',
     159: 'Touching (without moving) [part] of [something]',
     160: 'Trying but failing to attach [something] to [something] because it doesn’t stick',
     161: 'Trying to bend [something unbendable] so nothing happens',
     162: 'Trying to pour [something] into [something], but missing so it spills next to it',
     163: 'Turning [something] upside down',
     164: 'Turning the camera downwards while filming [something]',
     165: 'Turning the camera left while filming [something]',
     166: 'Turning the camera right while filming [something]',
     167: 'Turning the camera upwards while filming [something]',
     168: 'Twisting (wringing) [something] wet until water comes out',
     169: 'Twisting [something]',
     170: 'Uncovering [something]',
     171: 'Unfolding [something]',
     172: 'Wiping [something] off of [something]',
     173: '[Something] being deflected from [something]',
     174: '[Something] colliding with [something] and both are being deflected',
     175: '[Something] colliding with [something] and both come to a halt',
     176: '[Something] falling like a feather or paper',
     177: '[Something] falling like a rock'
}

## A) Model Instantiation

All **rtorchn** models can be imported from **rtorchn.core.networks**:

In [5]:
from rtorchn.core.networks import FullyConvolutionalNet

net = FullyConvolutionalNet(num_classes=178)

### Crash test

In [6]:
print('Default input size:', net.default_input_size())
# Create a dummy video
video = np.zeros(net.default_input_size(), "float32")
# Get prediction
pred = net.forward(Variable(torch.from_numpy(video)))
print('Prediction shape:', pred.size())

Default input size: (1, 3, 36, 96, 96)
Prediction shape: torch.Size([1, 178])


### Total number of parameters

In [7]:
num_params = sum([np.prod(param.size()) 
                  for param in net.parameters()])
print('Total number of parameters: {}'.format(num_params))

Total number of parameters: 8507554


## B) Loading weights

Send me a message if you want these weights

In [8]:
checkpoint = torch.load('/home/guillaume/weights/FullyConvolutionalNet/model.checkpoint')
net.load_state_dict(checkpoint)

## C) Testing 

In [9]:
TMP_VIDEO_FILENAME = 'tmp.mp4'

def unpreprocess(video):
    video = video.data.numpy()[0]
    video = 64. * video.transpose(1, 2, 3, 0)
    return np.array(video, 'uint8')

def demo(net, preprocessor, filename, top_n=5):
    # Open mpeg file and get a numpy array
    video_uint8 = open_mpeg_video(filename, 12, [128, 128])
    # Preprocessing
    video = preprocessor(video_uint8)
    # Convert to torch variable
    video = Variable(torch.from_numpy(video[None]), volatile=True)
    # Compute predictions
    pred = net.forward(video)
    # Convert to numpy 
    pred = np.exp(pred.data.numpy())[0]
    # Class index
    cls_indices = pred.argsort()[-top_n:][::-1]
    # Print class name with proba
    for index in cls_indices:
        cls = int2label[index]
        print('{:60s}{:.3f}'.format(cls, pred[index]))
    # Save input video in tmp file
    vwrite(TMP_VIDEO_FILENAME, unpreprocess(video))
    return show_video([filename, TMP_VIDEO_FILENAME])

def path_generator(annotation_path, root_path):
    with gzip.open(annotation_path, "rt") as f:
        annotations = json.load(f)
    files = [elem['file'] for elem in annotations]
    labels = [elem['template'] for elem in annotations]
    return ((os.path.join(root_path, f), label) for f,label in zip(files, labels))

In [10]:
# Path generator
path_gen = path_generator('/data/20bn-objects/json/test_20170627.json.gz', 
                          '/data/20bn-objects/videos')
# Put the network in evaluation mode
_ = net.eval()

#### Preprocessor

In [11]:
from rtorchn.preprocessing import default_evaluation_preprocesser

preprocessor = default_evaluation_preprocesser([48, 96, 96], 64.)

#### Ctrl + Enter

In [14]:
path_to_video, label = next(path_gen)
print('Label: {}\n{}'.format(label, 65*'-'))
demo(net, preprocessor, path_to_video)

Label: Throwing [something] onto a surface
-----------------------------------------------------------------
Pushing [something] so that it slightly moves               0.140
Letting [something] roll along a flat surface               0.113
Pushing [something] so that it almost falls off but doesn't 0.106
Rolling [something] on a flat surface                       0.101
Throwing [something]                                        0.023


## D) Feature Extraction

You can use the `extract_features` method to extract the activation values just before the output layers:

In [15]:
# Create a dummy video
video = np.zeros(net.default_input_size(), "float32")
# Get prediction
features = net.extract_features(Variable(torch.from_numpy(video)))
print('Prediction shape:', features.size())

Prediction shape: torch.Size([1, 4, 256])


## E) Finetuning

In [24]:
# Coming soon